# YOLOv12 Object Detection Training, Evaluation, and Inference

This notebook provides a professional pipeline for training, evaluating, and performing inference with a YOLOv12 model. It includes:
- Structured logging to track progress and errors
- Training and validation metrics visualization (loss, mAP) using Matplotlib from `results.csv`
- Comprehensive evaluation on validation and test sets
- Results storage in CSV format
- Model checkpointing and saving as `yolo12_trained.pt`
- Inference on a user-specified image with results overlaid and displayed in a new window
- Option to load a pre-trained model (`yolo12_trained.pt`) for inference

## Prerequisites
- Ensure `prepared_data.yaml` is correctly formatted with train/val/test paths for training/evaluation.
- Install required libraries: `pip install ultralytics matplotlib seaborn pandas pyyaml pillow`.
- GPU recommended for faster training/inference (set `device='cpu'` if GPU is unavailable).
- For inference, provide a path to a test image (e.g., `test_image.jpg`) in the inference cell.
- If using a pre-trained model, ensure `yolo12_trained.pt` exists in the specified path.


In [6]:
# Import necessary libraries
import os
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from ultralytics import YOLO
import logging
from datetime import datetime
import yaml
from PIL import Image
import subprocess

# Set up logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler(f'training_log_{datetime.now().strftime("%Y%m%d_%H%M%S")}.log'),
        logging.StreamHandler()
    ]
)
logger = logging.getLogger(__name__)

## Helper Functions

Define functions for loading data configuration, plotting training metrics, evaluating the model, and performing inference.

In [7]:
def load_data_config(data_yaml):
    """Load and validate the data configuration file."""
    try:
        with open(data_yaml, 'r') as f:
            config = yaml.safe_load(f)
        logger.info(f"Loaded data configuration from {data_yaml}")
        return config
    except Exception as e:
        logger.error(f"Failed to load data configuration: {e}")
        raise

def plot_training_metrics(save_dir):
    """Plot training and validation metrics from results.csv."""
    try:
        # Load metrics from results.csv
        results_csv_path = os.path.join(save_dir, 'yolo12_train', 'results.csv')
        if not os.path.exists(results_csv_path):
            logger.error(f"Results CSV not found at {results_csv_path}")
            return
        
        # Read the CSV file
        df = pd.read_csv(results_csv_path)
        
        # Strip whitespace from column names
        df.columns = df.columns.str.strip()
        
        # Create plots
        plt.style.use('seaborn')
        fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
        
        # Loss plot
        ax1.plot(df['train/box_loss'], label='Train Box Loss', color='blue')
        ax1.plot(df['val/box_loss'], label='Validation Box Loss', color='orange')
        ax1.set_title('Training and Validation Box Loss')
        ax1.set_xlabel('Epoch')
        ax1.set_ylabel('Loss')
        ax1.legend()
        ax1.grid(True)
        
        # mAP plot
        ax2.plot(df['metrics/mAP50(B)'], label='mAP@50', color='green')
        ax2.plot(df['metrics/mAP50-95(B)'], label='mAP@50:95', color='red')
        ax2.set_title('Validation mAP Metrics')
        ax2.set_xlabel('Epoch')
        ax2.set_ylabel('mAP')
        ax2.legend()
        ax2.grid(True)
        
        plt.tight_layout()
        plot_path = os.path.join(save_dir, 'training_metrics.png')
        plt.savefig(plot_path)
        plt.show()
        logger.info(f"Training metrics plot saved to {plot_path}")
    except Exception as e:
        logger.error(f"Failed to plot training metrics: {e}")

def evaluate_model(model, data_yaml, split='test'):
    """Evaluate the model on the specified dataset split."""
    logger.info(f"Evaluating model on {split} dataset...")
    results = model.val(data=data_yaml, split=split)
    metrics = {
        'mAP50': results.box.map50,
        'mAP50-95': results.box.map,
        'Precision': results.box.p,
        'Recall': results.box.r
    }
    logger.info(f"{split.capitalize()} Metrics: {metrics}")
    return metrics

def perform_inference(model, image_path, save_dir):
    """Perform inference on a single image and display results in a new window."""
    try:
        logger.info(f"Performing inference on {image_path}")
        # Run inference and save annotated image
        results = model.predict(image_path, save=True, save_dir=save_dir)
        
        # Get the path to the saved annotated image
        result_image_path = os.path.join(save_dir, os.path.basename(image_path))
        if not os.path.exists(result_image_path):
            logger.error(f"Annotated image not found at {result_image_path}")
            return
        
        # Open the annotated image in the system's default image viewer
        Image.open(result_image_path).show()
        logger.info(f"Inference result saved and displayed from {result_image_path}")
        
        # Log detection details
        for result in results:
            boxes = result.boxes
            logger.info(f"Detected {len(boxes)} objects: {boxes.cls}")
    except Exception as e:
        logger.error(f"Inference failed: {e}")
        raise

## Training and Evaluation Pipeline

Configure and run the training pipeline (if no pre-trained model is found), followed by evaluation. If a pre-trained model exists, skip training and proceed to inference.

In [ ]:
# Configuration
model_name = "yolo12n.pt"
pretrained_model_path = "yolo12_trained.pt"
data_yaml = "prepared_data.yaml"
epochs = 100
img_size = 1200
batch_size = 3
device = 0  # Set to 'cpu' if GPU is unavailable
save_dir = f'runs/train/exp_{datetime.now().strftime("%Y%m%d_%H%M%S")}'

# Create save directory
os.makedirs(save_dir, exist_ok=True)

try:
    # Check for pre-trained model
    if os.path.exists(pretrained_model_path):
        logger.info(f"Loading pre-trained model from {pretrained_model_path}")
        model = YOLO(pretrained_model_path)
    else:
        logger.info(f"No pre-trained model found at {pretrained_model_path}. Starting training...")
        # Load and validate data configuration
        data_config = load_data_config(data_yaml)
        
        # Initialize model
        logger.info(f"Loading YOLO model: {model_name}")
        model = YOLO(model_name)
        
        # Train model
        logger.info("Starting training...")
        results = model.train(
            data=data_yaml,
            epochs=epochs,
            imgsz=img_size,
            batch=batch_size,
            device=device,
            project=save_dir,
            name='yolo12_train',
            plots=True,
            save_period=10,  # Save checkpoint every 10 epochs
            patience=20,     # Early stopping if no improvement
            verbose=True
        )
        
        # Plot training metrics
        plot_training_metrics(save_dir)
        
        # Evaluate on validation set
        val_metrics = evaluate_model(model, data_yaml, split='val')
        
        # Evaluate on test set if available
        if 'test' in data_config:
            test_metrics = evaluate_model(model, data_yaml, split='test')
            
            # Save results to CSV
            results_df = pd.DataFrame({
                'Split': ['Validation', 'Test'],
                'mAP50': [val_metrics['mAP50'], test_metrics['mAP50']],
                'mAP50-95': [val_metrics['mAP50-95'], test_metrics['mAP50-95']],
                'Precision': [val_metrics['Precision'], test_metrics['Precision']],
                'Recall': [val_metrics['Recall'], test_metrics['Recall']]
            })
            results_path = os.path.join(save_dir, 'evaluation_results.csv')
            results_df.to_csv(results_path, index=False)
            logger.info(f"Evaluation results saved to {results_path}")
        
        # Save final model
        model.save(os.path.join(save_dir, 'yolo12_trained.pt'))
        logger.info(f"Final model saved to {save_dir}/yolo12_trained.pt")

except Exception as e:
    logger.error(f"Pipeline failed: {e}")
    raise

2025-09-24 14:05:02,620 - INFO - No pre-trained model found at yolo12_trained.pt. Starting training...
2025-09-24 14:05:02,631 - INFO - Loaded data configuration from prepared_data.yaml
2025-09-24 14:05:02,631 - INFO - Loading YOLO model: yolo12n.pt
2025-09-24 14:05:02,679 - INFO - Starting training...


Ultralytics 8.3.203  Python-3.11.9 torch-2.10.0.dev20250923+cu128 CUDA:0 (NVIDIA GeForce RTX 5060 Ti, 16311MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=6, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=prepared_data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=1, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1200, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo12n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=yolo12_train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=20, perspective=0.

## Inference on a Single Image

Provide the path to a test image below, and the model will perform inference, overlaying detection results (bounding boxes, labels) on the image and displaying it in a new window using the system's default image viewer.

In [ ]:
# Inference configuration
test_image_path = "test_image.jpg"  # Replace with your test image path

# Perform inference
if os.path.exists(test_image_path):
    perform_inference(model, test_image_path, save_dir)
else:
    logger.warning(f"Test image {test_image_path} not found. Please provide a valid image path.")

2025-09-24 14:01:17,100 - WARNING - Test image test_image.jpg not found. Please provide a valid image path.


## Results

The results are saved in the `runs/train/exp_YYYYMMDD_HHMMSS` directory (if training was performed). Check the following:
- `training_log_YYYYMMDD_HHMMSS.log`: Detailed training/inference logs
- `training_metrics.png`: Training and validation loss/mAP plots (if trained)
- `evaluation_results.csv`: Validation and test metrics (if trained and evaluated)
- `yolo12_trained.pt`: Trained model weights (if trained)
- Annotated inference image (e.g., `test_image.jpg`) in the save directory

If a pre-trained model was used, only inference results and logs are generated.
